<a href="https://colab.research.google.com/github/HarishSingh1981/SchoolOfAI_Assignment2.5/blob/main/Session2_5_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Two kind of inputs to the network 
#1. MNist dataset
#2. Random number
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.nn.init as init
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
import matplotlib.pyplot as plt

class MyDataset(Dataset):
  def __init__(self):
    super().__init__()
    self.image_dataset = torchvision.datasets.MNIST(root ="./data",
                                              train=True,
                                              download=True,
                                              transform=transforms.Compose([transforms.ToTensor()]))
    print(f'Length of Mnist dataset -- > {len(self.image_dataset)}')
  def __len__(self):
    return len(self.image_dataset)
  def __getitem__(self,index):
    #get image and label touple from data set + random number and summation of image label
    random_no = np.random.randint(0,9)
    sum_rand_label = random_no + self.image_dataset[index][1]
   # print(f'label for image data is {self.image_dataset[index][1]} and random number --> {random_no}')
    #touple2 = (random_no,sum_rand_label)
    rand_input = torch.tensor(np.zeros(10,dtype=np.float32),device= "cuda" if torch.cuda.is_available() else "cpu")
    rand_sum = torch.tensor(np.zeros(19,dtype=np.float32),device="cuda" if torch.cuda.is_available() else "cpu")
    rand_input[random_no] = 1
    rand_sum[sum_rand_label] = 1
    rand_input.requires_grad = True
    rand_sum.requires_grad = True
    #print(f'Random input {random_no} --> {rand_input}')
    #print(f'Random sum {sum_rand_label} --> {rand_sum}')
    #rand_arr = np.array([random_no],dtype=np.uint8)
    #sum_arr = np.array([sum_rand_label],dtype=np.uint8)
   # print(f'{rand_arr} .....{sum_arr}')
    #rand_arr = np.unpackbits(rand_arr)
    #sum_arr = np.unpackbits(sum_arr)
    #rand_arr = rand_arr[3:8]
    #sum_arr = sum_arr[3:8]
    #touple2 = (rand_arr.astype(dtype=np.float32),sum_arr.astype(dtype=np.float32))
   # print(f'rand_array--> {rand_arr} sum_array--> {sum_arr} touple2 --> {type(touple2)}')
    touple2 = (rand_input,rand_sum)
    return self.image_dataset[index],touple2

dataset = MyDataset()
#print(f'type of dataset is {type(dataset)}')
#hi
batch_size = 50
#myData_iter = iter(dataset)
#myData = next(myData_iter)
#print(f'Shape of data[0] and data[1]--> {type(myData[0])} ... {type(myData[1])}')
#print(f'{myData[0]} {myData[1]}')
#image,label = myData[0]
#number = myData[1]
#print(f'label,number --> {label},{number}')
myDataLoader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)
#print(f'length of dataset --> {len(myDataSet)} and type --> {type(myDataSet)}')

class MyNetwork(nn.Module):
  def __init__(self,batch_size):
    super().__init__()
    #assuming we are using Mnist data set with images 28*28
    self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
    #28*28 image will convolve to 24*24
    self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
    #28*28 image will convolve to 20*20
    self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
    self.fc2 = nn.Linear(in_features=120,out_features=60)
    self.out1 = nn.Linear(in_features=60,out_features=10)
    #Add one more fully connected layer of input 10+4 and output 10+5 neurons
    self.out2 = nn.Linear(in_features=10+10,out_features=19)

    self.batch_size = batch_size
  def forward(self,t1,t2):
    #input to convolution operation in pytorch is batch_size,input_channel,height,width
    x = t1
    x = self.conv1(x) #28x28 --> 24x24
    #visualize the learning with convolution
    x = F.relu(x)
    x = F.max_pool2d(x,kernel_size=2,stride=2) #half the size to 24x24 --> 12x12
    #visualize the effect of max_pool does it good to use it now?
    x = self.conv2(x) # 12x12 --> 8x8
    x = F.relu(x)
    x = F.max_pool2d(x,kernel_size=2,stride=2) #8x8 --> 4x4
    #print(f'shape of sample data after relu --> {x.shape}')
    x = x.reshape(self.batch_size,-1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.out1(x)
    #output one to predict image label
    output1 = F.softmax(x,dim=1)
    #output 2 to predict the sum, concatenate the neurons
    #print(f'shape of x --> {x.shape} and shape of t2 --> {t2.shape}')
    x = torch.cat([x,t2],dim=1)
    output2 = self.out2(x)
    output2 = F.softmax(output2,dim=1)
    return (output1,output2)

def weight_init(m):
  if isinstance(m,nn.Linear):
    init.uniform_(m.weight,-1,1)
    init.uniform_(m.bias,-1,1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Available device -- > {device}')
model = MyNetwork(batch_size)
#schedule the model to run on GPU
#print(f'gradient for conv1 {model.conv1.weight.grad}')
#images,labels = next(iter(myDataLoader))
#got the forward propogation and prediction
optimizer = optim.SGD(model.parameters(),lr=0.1)
#model.apply(weight_init)

model.requires_grad_(True)
#adding epoch also now
for epochs in range(100):
  total_loss_mnist = 0
  total_loss_sum = 0
  total_correct_mnist = 0
  total_correct_sum = 0
  for batchs in myDataLoader:
    #optimizer.zero_grad()
    #print(f'type of MyDataset -- > {type(MyDataset)}')
    #next_data = next(myData_iter)
    images,labels_mnist = batchs[0]
    #print(f'shape of labels_mnist is {labels_mnist.shape} and shape of image is {images.shape}')
    #print(f'type of labels_mnist is {type(labels_mnist)} and shape of image is {type(images)}')
    #visualization of images of Mnist dataset
    #print(f'Shape of images -- > {images.shape}')
    #grid = torchvision.utils.make_grid(images,nrow=10)
    #plt.figure(figsize=[15,15])
    #reordering image dimensions
    #plt.imshow(grid.permute(1,2,0))
    #labels = torch.tensor(labels_mist)
    #print(f'type of batchs and batch[0] and batch[1]--> {type(batchs)}, {type(batchs[0])}, {type(batchs[1])}')
    #print(f'len of batchs and batch[0] and batch[1]--> {len(batchs)}, {len(batchs[0])}, {len(batchs[1])}')
    #print(f'{batchs[0]}, {batchs[1]}')
    random_num,label_sum = batchs[1]

    #print(f'shape of random_num is {random_num.shape} and shape of label_sum is {label_sum.shape}')
    #print(f'type of random_num is {type(random_num)} and shape of label_sum is {type(label_sum)}')

    #print(f'Shape of images --> {images.shape} and labels_mist --> {labels_mist.shape}')
    preds_mnist,preds_sum = model(images,random_num)
    #find the error
    #print(f'preds_mnist --> {preds_mnist} and preds_sum --> {preds_sum} and label_sum {label_sum}')
    loss_mnist = F.cross_entropy(preds_mnist,labels_mnist)
    loss_sum = F.cross_entropy(preds_sum,label_sum)
    #It does not take 2 outputs
    #loss = F.cross_entropy((preds_mnist,preds_sum),(labels_mist,label_sum))
    #calculate average mean loss
    avg_loss = 0.5*loss_mnist+0.5*loss_sum
    #calculate the gradient
    avg_loss.backward()
    #print(f'gradient for conv1 {model.conv1.weight.grad}')
    #update the weights
    optimizer.step()
    #print(f'loss --> {loss} and values --> {pred.argmax(dim=1).eq(label).sum().item()}')
    optimizer.zero_grad()
    total_loss_mnist += loss_mnist
    total_loss_sum += loss_sum
    #print(f'Shape of pred --> {preds.shape} and labels --> {labels.shape}')
    total_correct_mnist += preds_mnist.argmax(dim=1).eq(labels_mnist).sum().item()
    #print(f'shape of preds_mnist {preds_mnist.shape} and shape of preds_sum {preds_sum.shape}')
    #print(f'shape of label_sum is {label_sum.shape} and shape of lables_mist {labels_mnist.shape}')
    total_correct_sum += preds_sum.argmax(dim=1).eq(label_sum.argmax(dim=1)).sum().item()
    #print(f'{model.out2.state_dict()}')
  print(f'epoch --> {epochs} loss_mnist --> {total_loss_mnist} and correct predictions --> {total_correct_mnist}')
  print(f'epoch --> {epochs} loss_sum --> {total_loss_sum} and correct predictions of sum --> {total_correct_sum}')